<a href="https://colab.research.google.com/github/Mathnar/Mean_shift_clustering/blob/CUDA_colab/Parallel_CUDA_meanshift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Controllo funzioni GPU - CUDA**

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-xf73qwz7
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-xf73qwz7
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4307 sha256=6c740bc851028500771a6a644b580efd9da2ac3a04d3b57345688304f391a037
  Stored in directory: /tmp/pip-ephem-wheel-cache-i_cou_im/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [ ]:
import torch
torch.cuda.is_available()
import tensorflow as tf
tf.test.gpu_device_name()
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Mon Nov 21 17:09:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |    816MiB / 15109MiB |      0%      Default |
|                               |      

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <cstdio>
#include <iostream>

using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block;
	block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];
	
	for (int i = 0; i < n; i++) 
 	{
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}

The maximum element is : -1496946672
The time required : 0.003392



**CUDA_mean_shift**

---



---



In [ ]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%cu  %%cuda  %%cuda_run  %%debug  %%file  %%html  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%cu

#include <vector>
#include <iostream>
#include <fstream>
#include <sstream>

std::vector<float> get_data(const std::string& file){
    std::vector<float> x;
    std::vector<float> y;
    std::vector<float> z;
    std::ifstream data(file);
    std::string line;
    while(std::getline(data, line)){
    	std::vector<float> coords;
        std::stringstream row(line);
        std::string el;
        while(std::getline(row, el, ',')){
            coords.push_back(stod(el));
        }
        x.push_back(coords[0]);
        y.push_back(coords[1]);
        z.push_back(coords[2]);
    }
    std::vector<float> points = x;
    points.insert(points.end(), y.begin(), y.end());
    points.insert(points.end(), z.begin(), z.end());
    return points;
}


#include "/content/drive/My Drive/CUDA_mean_shift/math.h"
#include <iostream>
#include <numeric>
#include <stdlib.h>
#include <chrono>

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/sort.h>


#define BDW 2
#define N_DIM 3
#define NUM_ITERATIONS 5
#define NUM_TESTS 15
#define BLOCK_DIM 64


__global__ void CUDA_mean_shift(float* movedPoints, const float* zeroPoints, const unsigned nPoints) {
	int idx = blockIdx.x * blockDim.x + threadIdx.x;
	float3 newCoords = make_float3(0.0, 0.0, 0.0);
	float totalWeight = 0.0;

	if(idx < nPoints){
		float x = movedPoints[idx];
		float y = movedPoints[idx + nPoints];
		float z = movedPoints[idx + 2 * nPoints];
		float3 movedPoint = make_float3(x, y, z);

		for(int i = 0; i < nPoints; i++){
			x = zeroPoints[i];
			y = zeroPoints[i + nPoints];
			z = zeroPoints[i + 2 * nPoints];
			float3 zeroPoint = make_float3(x, y, z);
			float3 difference = movedPoint - zeroPoint;
			float squaredDistance = dot(difference, difference);
			float weight = std::exp((-squaredDistance) / (2 * powf(BDW, 2)));
			newCoords += zeroPoint * weight;
			totalWeight += weight;
		}
		newCoords /= totalWeight;
		movedPoints[idx] = newCoords.x;
		movedPoints[idx + nPoints] = newCoords.y;
		movedPoints[idx + 2 * nPoints] = newCoords.z;
	}
}

int main(void)
{

	std::string data = "/content/drive/My Drive/CUDA_mean_shift/data_250k.csv";
	std::vector<float> points = get_data(data);

	int nPoints = points.size() / N_DIM;
	printf("Testing--> --> point to be clustered: %d\n", nPoints);

	float timer = 0.0;

	thrust::device_vector<float> zeroPoints = points;
	thrust::device_vector<float> movedPoints = points;

	timer = 0.0;

	for(int j = 0; j < NUM_TESTS; j++){
			zeroPoints = points;
			movedPoints = points;

			float* zeroPoint = thrust::raw_pointer_cast(&zeroPoints[0]);
			float* movedPoint = thrust::raw_pointer_cast(&movedPoints[0]);

			dim3 gridDim = dim3(ceil((float) nPoints / BLOCK_DIM)); //variare block dim e bdw in base al blocco migliore
			dim3 blockDim = dim3(BLOCK_DIM);

			std::chrono::high_resolution_clock::time_point start = std::chrono::high_resolution_clock::now();
			for(int i = 0; i < NUM_ITERATIONS; i++){
				CUDA_mean_shift<<<gridDim, blockDim>>> (movedPoint, zeroPoint, nPoints);
				cudaDeviceSynchronize();
			}
			std::chrono::high_resolution_clock::time_point end = std::chrono::high_resolution_clock::now();
			float elapsedTime = std::chrono::duration_cast<std::chrono::duration<float>>(end - start).count();
			timer += elapsedTime;
	}
	timer /= NUM_TESTS;
	printf("\nCUDA Mean shift clustering \nTot Time of EXE %f\n", timer);

	return 0;
}


Testing--> --> point to be clustered: 250000

CUDA Mean shift clustering 
Tot Time of EXE 2.685420

